In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# .env 파일에 LANGCHAIN_API_KEY를 입력합니다.
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

# LangSmith 추적을 원하지 않을 경우
# logging.langsmith("CH01-Basic", set_enable=False)

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


### 데이터를 효과적으로 전달하는 방법
- `RunnalbePassthrough`는 입력을 변경하지 않거나 추가 키를 더하여 전달할 수 있습니다.
- `RunnablePassthrough()`가 단독으로 호출되면, 단순히 입력을 받아 그대로 전달합니다.
- `RunnablePassthrough.assign(...)`방식으로 호출되면, 입력을 받아 assign 함수에 전달된 추가 인수를 추가합니다.

### RunnablePassthrough

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# prompt와 llm을 생성합니다.
prompt = PromptTemplate.from_template("{num}의 10배는?")
llm = ChatOpenAI()

# chain을 생성합니다.
chain = prompt | llm

chain을 `invoke()`하여 실행할 때는 입력 데이터의 타입이 딕셔너리여야 합니다.

In [4]:
# chain을 실행합니다.
chain.invoke({"num" : 5})

AIMessage(content='50', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CqJMt50gCIVVQnCqaXxE7T4zRpikY', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--61f54aaf-96c2-42d8-a1e5-3d4c665045d5-0', usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

하지만, langchain 라이브러리가 업데이트 되면서 1개의 변수만 템플릿에 포함되고 있다면, 값만 전달하는 것도 가능합니다.

In [5]:
chain.invoke(5)

AIMessage(content='50', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CqJNhZpFWwqj4Gpxj2C4yGdkCqWdG', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d3fa3797-18d7-466b-9b41-b7ce86ee73e8-0', usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

아래는 `RunnablePassthrough`를 사용한 예제입니다. </br>

`RunnablePassthrough`는 `runnable`객체이며, `runnable`객체는 `invoke()`메서드를 사용하여 별도 실행이 가능합니다.

In [6]:
from langchain_core.runnables import RunnablePassthrough

# runnable
RunnablePassthrough().invoke({"num" : 10})

{'num': 10}

아래는 `RunnablePassthrough`로 체인을 구성하는 예제입니다.

In [7]:
runnable_chain = {"num" : RunnablePassthrough()} | prompt | ChatOpenAI()

# dict 값이 RunnablePassthrough()로 변경되었습니다.
runnable_chain.invoke(10)

AIMessage(content='100', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CqJQqtJhcza2LRzKntoIOQguj9Muu', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--69537184-662a-4bbd-8ef0-ab9a0bdcfad5-0', usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

다음은 `RunnablePassthrough.assign()`을 사용하는 경우와 비교한 결과입니다.

In [8]:
RunnablePassthrough().invoke({"num" : 10})

{'num': 10}

`RunnablePassthrough.assign()`
- 입력 값으로 들어온 값의 key/value 쌍과 새롭게 할당된 key/value 쌍을 합칩니다.

In [9]:
# 입력 키 : num, 할당(assign) 키 : new_num
(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num" : 1})

{'num': 1, 'new_num': 3}

### RunnableLambda
RunnableLambda를 사용하여 사용자 정의 함수를 맵핑할 수 있습니다.

In [18]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from datetime import datetime

def get_today(a):
    print(f"입력받은 변수 a의 값 : {a}")
    # 오늘 날짜를 가져오기
    return datetime.today().strftime("%b-%d")

# 오늘 날짜를 출력
get_today(None)

입력받은 변수 a의 값 : None


'Dec-25'

In [8]:
"""
왜 get_today() 함수에 의미 없는 매개변수를 넣었을까?
RunnableLambda로 만든 chain을 invoke할 때,
사용하는 값이 RunnableLambda로 감싼 함수의 매개변수로 들어가기 때문에
무조건 매개변수가 하나는 필요하다. 매개변수가 없으면 에러가 발생한다.
"""

'\n왜 get_today() 함수에 의미 없는 매개변수를 넣었을까?\nRunnableLambda로 만든 chain을 invoke할 때,\n사용하는 값이 RunnableLambda로 감싼 함수의 매개변수로 들어가기 때문에\n무조건 매개변수가 하나는 필요하다. 매개변수가 없으면 에러가 발생한다.\n'

In [19]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# prompt와 llm을 생성합니다.
prompt = PromptTemplate.from_template(
    "{today}가 생일인 유명인 {n}명을 나열하세요. 생년월일을 표기해 주세요."
)

llm = ChatOpenAI(temperature=0)

# chain을 생성합니다.
chain = (
    {"today" : RunnableLambda(get_today), "n" : RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [20]:
# 출력
print(chain.invoke(3))

입력받은 변수 a의 값 : 3
1. 크리스마스 생일인 유명인 3명:
- 세이지 브룬스 (Sissy Spacek) - 1949년 12월 25일
- 카밀라 카베요 (Camila Cabello) - 1997년 12월 25일
- 안토니오 곤자레스 (Antonio Gonzalez) - 1996년 12월 25일


`itemgetter`를 사용하여 특정 키를 추출합니다.

In [21]:
from operator import itemgetter

# chain을 생성합니다.
chain1 = (
    {"today" : RunnableLambda(get_today), "n" : itemgetter("n")}
    | prompt
    | llm
    | StrOutputParser()
)

In [22]:
# 출력
print(chain1.invoke({"n" : 3}))

입력받은 변수 a의 값 : {'n': 3}
1. 크리스마스 생일인 유명인 3명:
- 칼 긱슨 (1924년 12월 25일)
- 안토니오 산타 엘리아 (1971년 12월 25일)
- 저스틴 채트윈 (1981년 12월 25일)


In [25]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda

from operator import itemgetter

# 문장의 길이를 반환하는 함수입니다.
def length_function(text):
    return len(text)

# 두 문장의 길이를 곱한 값을 반환하는 함수입니다.
def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)

# _multiple_length_function 함수를 사용하여 두 문장의 길이를 곱한 값을 반환하는 함수입니다.
def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])

prompt = ChatPromptTemplate.from_template("{a} + {b}는 무엇인가요?")
model = ChatOpenAI()

chain1 = prompt | model

chain = (
    {
        "a" : itemgetter("word1") | RunnableLambda(length_function),
        "b" : {"text1" : itemgetter("word1"), "text2" : itemgetter("word2")} | RunnableLambda(multiple_length_function)
    }
    | prompt
    | model
    | StrOutputParser()
)

In [26]:
chain.invoke({"word1" : "hello", "word2" : "world"})

'5 + 25는 30 입니다.'